### 1. 선형 회귀 모델

: 종속변수와 독립변수 간의 상관관계를 모델링하는 방법

    (선형 방정식으로 표현해 예측할 데이터를 분류하는 모델 -> 선형으로 분리할 수 있음 (차원과 관계없음))
    
    - 2차원 : 단항 회귀 모델 vs 3차원 ~ : 다항 회귀 모델


### 1.1 로지스틱 회귀 모델

: 선형 모델의 결괏값에 로지스틱 함수를 적용해 0 ~ 1 사이의 값을 갖게 해서 확률로 표현한다. -> 2개로 분류

    (1에  가까우면 정답이 1이라고 예측하고, 0에 가까운 경우 0으로 예측한다.

## 4.1.4 Logistic Regression Example with TF-IDF

### TF-IDF Feature Example

- TF-IDF 활용해서 문장 벡터 만들기

In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [3]:
train_data = pd.read_csv( DATA_IN_PATH + TRAIN_CLEAN_DATA )

In [4]:
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

#### TF-IDF 벡터화

In [5]:
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3), max_features=5000) 
# min_df: 설정한 값보다 특정 토큰의 df 값(자유도)이 더 적게 나오면 벡터화 과정에서 제거한다는 의미
# anlyzer: 분석하기 위한 기준 단위 (char: 문자단위 / word: 단어단위)
# sublinear_tf: 문서의 단어 빈도에 대한 스무딩 여부를 설정하는 값
# ngram_range: 빈도의 기본단위를 어느 범위의 n-gram으로 설정할 것인지를 보는 인자
# max_features: 각 벡터의 최대 길이, 특징의 길이를 설정하는 것

X = vectorizer.fit_transform(reviews)
y = np.array(sentiments)

In [25]:
print(X, y)

  (0, 3881)	0.04506147239359123
  (0, 4129)	0.04607784447281331
  (0, 4337)	0.04001308835469651
  (0, 1493)	0.03314495265227629
  (0, 0)	0.054660407781216024
  (0, 1619)	0.040403370498837535
  (0, 3083)	0.046079687549730936
  (0, 1951)	0.04618604633963843
  (0, 2804)	0.04640415994477519
  (0, 2627)	0.04304829623020376
  (0, 1153)	0.051684947322936506
  (0, 2225)	0.04703917716139301
  (0, 294)	0.046598741768661085
  (0, 3572)	0.043470981570003044
  (0, 958)	0.04078225166901166
  (0, 2396)	0.04529752186392342
  (0, 772)	0.04208928002687366
  (0, 4640)	0.036067513138924284
  (0, 1776)	0.038328017500383296
  (0, 4834)	0.03836482875414165
  (0, 4949)	0.015777811576590577
  (0, 2277)	0.03402898804011063
  (0, 614)	0.034619389056850976
  (0, 3402)	0.037875470600814684
  (0, 4562)	0.03001653177274675
  :	:
  (24999, 2043)	0.04848474157651679
  (24999, 2347)	0.06978183450498811
  (24999, 1320)	0.05159234341989928
  (24999, 3617)	0.05505645496853158
  (24999, 4544)	0.0403923682938471
  (24999, 2

In [7]:
features = vectorizer.get_feature_names()

#### 학습과 검증 데이터셋 분리

In [8]:
X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)
# train 데이터 중에 validation 데이터를 만듬

#### 모델 선언 및 학습

In [9]:
lgs = LogisticRegression(class_weight='balanced')  
# The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y)).
lgs.fit(X_train, y_train)

c:\users\nsa18\anaconda3\envs\ml_nlp\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

#### 검증 데이터으로 성능 평가

In [10]:
predicted = lgs.predict(X_eval)

In [11]:
print("Accuracy: %f" % lgs.score(X_eval, y_eval)) # 튜닝 전, 하나의 기준 지표가 될 수 있다.

Accuracy: 0.859600


#### 데이터 제출하기

In [12]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

In [13]:
# 학습 데이터에 대해 사용했던 객체를 사용해 TF-IDF 값으로 벡터화한다.
testDataVecs = vectorizer.transform(test_data['review'])

In [14]:
test_predicted = lgs.predict(testDataVecs)
print(test_predicted)

[1 0 1 ... 0 1 0]


In [22]:
test_data.review[0], test_data.review[1], test_data.review[2] 

('naturally film main themes mortality nostalgia loss innocence perhaps surprising rated highly older viewers younger ones however craftsmanship completeness film anyone enjoy pace steady constant characters full engaging relationships interactions natural showing need floods tears show emotion screams show fear shouting show dispute violence show anger naturally joyce short story lends film ready made structure perfect polished diamond small changes huston makes inclusion poem fit neatly truly masterpiece tact subtlety overwhelming beauty',
 'movie disaster within disaster film full great action scenes meaningful throw away sense reality let see word wise lava burns steam burns stand next lava diverting minor lava flow difficult let alone significant one scares think might actually believe saw movie even worse significant amount talent went making film mean acting actually good effects average hard believe somebody read scripts allowed talent wasted guess suggestion would movie start 

In [23]:
if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)

answer_dataset = pd.DataFrame({'id': test_data['id'], 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_tfidf_answer.csv', index=False, quoting=3)